# Biocon DP Demand Planning & BOM Analysis
## Multi-File Integration: Demand Mapping & Procurement Analysis

**Purpose:** Extract L2 demand, map to BOM, calculate procurement needs

**Files Used:**
1. Aug'25_L2_DP_Plan_Circulation_V2.xlsx - L2 demand plan
2. ParkourSC_SNP.xlsx - Supply chain planned production
3. 20251006-DP Material Shortage - Working file.xlsx - BOM/Materials

**Output:** Excel file with 9 sheets of analysis

## Step 1: Import Libraries & Setup

In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils import get_column_letter
from datetime import datetime
import warnings
import re

warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✓ Libraries imported successfully
Pandas version: 2.3.3
NumPy version: 2.3.4
Timestamp: 2025-11-06 12:27:31


## Step 2: Configure File Paths & Settings

In [2]:
# File Paths - UPDATE THESE IF FILES ARE IN DIFFERENT LOCATIONS
AUG25_FILE = "/home/supriyo/Downloads/Biocon_nw/Aug'25_L2_DP_Plan_Circulation_V2.xlsx"
SNP_FILE = "/home/supriyo/Downloads/Biocon_nw/ParkourSC_SNP.xlsx"
BOM_FILE = "/home/supriyo/Downloads/Biocon_nw/20251006-DP Material Shortage - Working file.xlsx"
OUTPUT_FILE = "/mnt/user-data/outputs/Demand_Planning_Analysis.xlsx"

# Sheet Names
AUG25_SHEET = "L2Ph1_Detail"
SNP_SHEET = "DP RCCP"
BOM_SHEET = "DP Shortage"

# Products Configuration
PRODUCTS_AVAILABLE = [700001012, 700001123, 700000536, 700001318, 700001301]
PRODUCTS_NOT_AVAILABLE = [700004130]

print("✓ Configuration loaded")
print(f"  Input files: 3")
print(f"  Output file: {OUTPUT_FILE}")
print(f"  Products to analyze: {len(PRODUCTS_AVAILABLE)}")

✓ Configuration loaded
  Input files: 3
  Output file: /mnt/user-data/outputs/Demand_Planning_Analysis.xlsx
  Products to analyze: 5


## Step 3: Define Product Hierarchy Mapping

In [3]:
# Define all SKU mappings with product hierarchy
mCB_skus = ['800004403', '800004402', '800008019', '800008020', '800008034', '800007997', '800007345', '800007516',
            '800002513', '800007608', '800007630', '800002984', '800004986', '800007310', '800007311', '800006648',
            '800007634', '800008073', '800006523', '800002297', '800002872', '800006741', '800007380']

sMCB_skus = ['800006506', '800006505', '800006527', '800006526', '800006525', '800007546', '800007583', '800007839',
             '800006524', '800006627', '800007872']

vial_skus = ['800004400', '800004401', '800006626', '800006740', '800007996']

aspart_dlp_skus = ['800008016', '800002958', '800002948', '800006528', '800002989', '800003528', '800006592', '800006691']

aspart_vial_skus = ['800008017', '800006529']

rhi_skus = ['800001300', '800001298', '800001299']

# Build product mapping
product_mapping = {}

for sku in mCB_skus:
    product_mapping[sku] = {'assembly': '700003964', 'filling': '700001012', 'root': '700001470', 'family': 'Glargine_mCB_DLP'}

for sku in sMCB_skus:
    product_mapping[sku] = {'assembly': '700004129', 'filling': '700004130', 'root': '700004130', 'family': 'Glargine_sMCB_DLP_EU'}

for sku in vial_skus:
    product_mapping[sku] = {'assembly': '700001123', 'filling': '700001123', 'root': '700001123', 'family': 'Glargine_Vial'}

for sku in aspart_dlp_skus:
    product_mapping[sku] = {'assembly': '700002770', 'filling': '700001301', 'root': '700001301', 'family': 'Aspart_DLP'}

for sku in aspart_vial_skus:
    product_mapping[sku] = {'assembly': '700001318', 'filling': '700001318', 'root': '700001318', 'family': 'Aspart_Vial'}

for sku in rhi_skus:
    product_mapping[sku] = {'assembly': '700000536', 'filling': '700000536', 'root': '700000536', 'family': 'RHI'}

print(f"✓ Product hierarchy defined with {len(product_mapping)} SKU mappings")
print(f"  - mCB: {len(mCB_skus)} SKUs → 700001012")
print(f"  - sMCB: {len(sMCB_skus)} SKUs → 700004130 (NOT AVAILABLE)")
print(f"  - Vial: {len(vial_skus)} SKUs → 700001123")
print(f"  - Aspart DLP: {len(aspart_dlp_skus)} SKUs → 700001301")
print(f"  - Aspart Vial: {len(aspart_vial_skus)} SKUs → 700001318")
print(f"  - RHI: {len(rhi_skus)} SKUs → 700000536")

✓ Product hierarchy defined with 52 SKU mappings
  - mCB: 23 SKUs → 700001012
  - sMCB: 11 SKUs → 700004130 (NOT AVAILABLE)
  - Vial: 5 SKUs → 700001123
  - Aspart DLP: 8 SKUs → 700001301
  - Aspart Vial: 2 SKUs → 700001318
  - RHI: 3 SKUs → 700000536


## Step 4: Helper Functions

In [4]:
def normalize_month(header_str):
    """Convert month header to standardized format (MM.YYYY)"""
    month_map = {
        'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
        'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
    }
    
    header_str = str(header_str).upper().strip()
    
    for month_str, month_num in month_map.items():
        if month_str in header_str:
            year_match = re.search(r'202\d', header_str)
            if year_match:
                year = year_match.group()
                return f"{month_num:02d}.{year}"
    
    return header_str

print("✓ Helper functions defined")

✓ Helper functions defined


## Step 5: Extract L2 Demand Data

In [5]:
print("Loading Aug25 L2 demand data...\n")

try:
    wb_aug25 = openpyxl.load_workbook(AUG25_FILE)
    ws_aug25 = wb_aug25[AUG25_SHEET]
    
    # Get month headers from columns S to AM (columns 19-39)
    month_headers = []
    for col_idx in range(19, 40):
        header = ws_aug25.cell(row=2, column=col_idx).value
        if header:
            month_headers.append(normalize_month(header))
    
    print(f"✓ Found {len(month_headers)} month headers")
    print(f"  Months: {month_headers[:3]} to {month_headers[-2:]}")
    
    # Extract demand data
    l2_demand_data = []
    
    for row_idx in range(3, 79):  # Row 3 to 78
        market_sku = ws_aug25.cell(row=row_idx, column=5).value  # Col E
        batch_size = ws_aug25.cell(row=row_idx, column=11).value  # Col K
        
        if market_sku and str(market_sku).strip() != '':
            market_sku = str(market_sku).strip()
            batch_size = float(batch_size) if batch_size else 0
            
            product_info = product_mapping.get(market_sku, {})
            
            # Extract monthly demand
            for col_idx, month in enumerate(month_headers):
                ea_col = 19 + col_idx  # Column S onwards
                batch_col = 40 + col_idx  # Column AN onwards
                
                ea_value = ws_aug25.cell(row=row_idx, column=ea_col).value
                batch_value = ws_aug25.cell(row=row_idx, column=batch_col).value
                
                ea_value = float(ea_value) if ea_value and str(ea_value).strip() != '' else 0
                batch_value = float(batch_value) if batch_value and str(batch_value).strip() != '' else 0
                
                l2_demand_data.append({
                    'Market_SKU': market_sku,
                    'Product_ID': product_info.get('filling', 'N/A'),
                    'Product_Family': product_info.get('family', 'Unknown'),
                    'Batch_Size': batch_size,
                    'Month': month,
                    'EA': ea_value,
                    'Batches': batch_value
                })
    
    df_l2_demand = pd.DataFrame(l2_demand_data)
    
    print(f"\n✓ Extracted L2 demand data")
    print(f"  Total records: {len(df_l2_demand)}")
    print(f"  Unique SKUs: {df_l2_demand['Market_SKU'].nunique()}")
    print(f"  Unique Products: {df_l2_demand['Product_ID'].nunique()}")
    print(f"  Date range: {df_l2_demand['Month'].min()} to {df_l2_demand['Month'].max()}")
    
    print("\nSample L2 Demand Data:")
    print(df_l2_demand.head(10))
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

Loading Aug25 L2 demand data...

✓ Found 21 month headers
  Months: ['10.2025', '11.2025', '12.2025'] to ['05.2027', '06.2027']
❌ Error: could not convert string to float: '=S3/$K3'


Traceback (most recent call last):
  File "/tmp/ipykernel_137597/3889469501.py", line 39, in <module>
    batch_value = float(batch_value) if batch_value and str(batch_value).strip() != '' else 0
                  ^^^^^^^^^^^^^^^^^^
ValueError: could not convert string to float: '=S3/$K3'


## Step 6: Extract SNP Planned Production

In [6]:
print("Loading SNP planned production data...\n")

try:
    df_snp = pd.read_excel(SNP_FILE, sheet_name=SNP_SHEET)
    
    print(f"✓ SNP data loaded")
    print(f"  Shape: {df_snp.shape}")
    print(f"  Columns: {list(df_snp.columns)}\n")
    
    # Filter for Planned Production
    df_snp_planned = df_snp[df_snp['Key Figure'] == 'Planned Production'].copy()
    print(f"✓ Filtered for 'Planned Production': {len(df_snp_planned)} records")
    
    # Convert Product ID to numeric
    product_col = [col for col in df_snp_planned.columns if 'Product' in col][0]
    df_snp_planned['Product_ID'] = pd.to_numeric(df_snp_planned[product_col], errors='coerce')
    
    # Filter for available products
    df_snp_filtered = df_snp_planned[df_snp_planned['Product_ID'].isin(PRODUCTS_AVAILABLE)].copy()
    
    print(f"✓ Filtered for available products: {len(df_snp_filtered)} records\n")
    print("Sample SNP Data:")
    print(df_snp_filtered.head())
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

Loading SNP planned production data...

✓ SNP data loaded
  Shape: (892, 35)
  Columns: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34']

❌ Error: 'Key Figure'


Traceback (most recent call last):
  File "/home/supriyo/repogen/.venv/lib/python3.11/site-packages/pandas/core/indexes/base.py", line 3812, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas/_libs/index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7096, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Key Figure'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/tmp/ipykernel_137597/2982042106.py", line 11, in <module>
    df_snp_planned = df_snp[df_snp['Key Figure'] == 'Planned Production'].copy()
                            ~~~~~~^^^^^^^^^^^^^^
  File "/home/supriyo/repogen/.venv/

## Step 7: Extract BOM Material Data

In [7]:
print("Loading BOM material data...\n")

try:
    wb_bom = openpyxl.load_workbook(BOM_FILE)
    ws_bom = wb_bom[BOM_SHEET]
    
    bom_data = []
    
    for row_idx in range(23, 543):  # Row 23 to 542
        material_id = ws_bom.cell(row=row_idx, column=1).value  # Col A
        material_desc = ws_bom.cell(row=row_idx, column=2).value  # Col B
        model = ws_bom.cell(row=row_idx, column=3).value  # Col C
        product_family = ws_bom.cell(row=row_idx, column=4).value  # Col D
        section = ws_bom.cell(row=row_idx, column=5).value  # Col E
        component_id = ws_bom.cell(row=row_idx, column=6).value  # Col F
        
        # Filter: skip blank and 0 values
        if material_id is None or str(material_id).strip() == '' or material_id == 0:
            continue
        
        bom_data.append({
            'Material_ID': material_id,
            'Material_Description': material_desc,
            'Model': model,
            'Product_Family': product_family,
            'Section': section,
            'Component_ID': component_id
        })
    
    df_bom = pd.DataFrame(bom_data)
    
    print(f"✓ Extracted BOM data")
    print(f"  Total materials: {len(df_bom)}")
    print(f"  Unique product families: {df_bom['Product_Family'].nunique()}\n")
    
    print("Sample BOM Data:")
    print(df_bom.head(10))
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

Loading BOM material data...

✓ Extracted BOM data
  Total materials: 520
  Unique product families: 16

Sample BOM Data:
  Material_ID                      Material_Description  \
0   300000053  EMFLOW FILTER-BROWN(Cat No:AB05PFR2PVH4)   
1   300000068                     MILLIPAK200 MPGL2GCA3   
2   300000072            NOVASIP FILTER(Cat No:C3PFRP1)   
3   300000121  "PALL EMFLON - 10 "" (Cat No:AB1PFR7PVH4   
4   300000168                     MILLIPAK100 MPGL1GCA3   
5   300001482         FILTER 4'' #MIL0001L1462176 MERCK   
6   300004220  FILTER 4" SURF ASSEMBLY MIL0001L2302167    
7   300001483        FILTER 10'' #MIL0001L1462177 MERCK   
8   300001573    Polysep II Cartridge Filter #CGW371S03   
9   300003673   Polysep II Cartridge Filter20"CGW372S03   

                                  Model Product_Family          Section  \
0  R_N_30/70_GC_GV__AC_AV_B; LN2 - R_GC            RGA    01-Filling_01   
1                                    GV              G  01-Filling_Vial   
2  

## Step 8: Create Summary Sheets

In [8]:
print("Creating summary sheets...\n")

# 1. L2 Demand Summary (by Month and Product)
df_l2_summary = df_l2_demand.groupby(['Month', 'Product_ID', 'Product_Family']).agg({
    'EA': 'sum',
    'Batches': 'sum',
    'Market_SKU': 'count'
}).rename(columns={'Market_SKU': 'SKU_Count'}).reset_index()

print(f"✓ L2 Demand Summary: {len(df_l2_summary)} records")

# 2. Product Family Summary
df_family_summary = df_l2_demand.groupby('Product_Family').agg({
    'EA': 'sum',
    'Batches': 'sum',
    'Market_SKU': 'nunique',
    'Product_ID': 'nunique'
}).rename(columns={
    'EA': 'Total_EA',
    'Batches': 'Total_Batches',
    'Market_SKU': 'Unique_SKUs',
    'Product_ID': 'Unique_Products'
}).reset_index()

print(f"✓ Product Family Summary: {len(df_family_summary)} families")

# 3. BOM Summary by Product Family
df_bom_summary = df_bom.groupby('Product_Family').agg({
    'Material_ID': 'count',
    'Component_ID': 'nunique'
}).rename(columns={
    'Material_ID': 'Total_Materials',
    'Component_ID': 'Unique_Components'
}).reset_index()

print(f"✓ BOM Summary: {len(df_bom_summary)} families")

# 4. Procurement Analysis
procurement_data = []
for _, row in df_snp_filtered.iterrows():
    procurement_data.append({
        'Product_ID': row.get('Product_ID', 'N/A'),
        'Total_Demand': 0,
        'Unrestricted_Stock': 0,
        'OPO': 0,
        'OS_PR': 0,
        'Safety_Stock': 0
    })

df_procurement = pd.DataFrame(procurement_data)
df_procurement['Procurement_Upper_Bound'] = (
    df_procurement['Total_Demand']
    - (
        df_procurement['Unrestricted_Stock']
        + df_procurement['OPO']
        + df_procurement['OS_PR']
        - df_procurement['Safety_Stock']
    )
).clip(lower=0)

print(f"✓ Procurement Analysis: {len(df_procurement)} products")

# 5. Product Coverage
products_in_demand = set(df_l2_demand['Product_ID'].unique())
products_in_snp = set(df_snp_filtered['Product_ID'].unique())
products_coverage = products_in_demand.union(products_in_snp)

df_coverage = pd.DataFrame({
    'Product_ID': list(products_coverage),
    'In_L2_Demand': [pid in products_in_demand for pid in products_coverage],
    'In_SNP_Plan': [pid in products_in_snp for pid in products_coverage]
})

print(f"✓ Product Coverage: {len(df_coverage)} products\n")

print("Summary creation complete!")

Creating summary sheets...



NameError: name 'df_l2_demand' is not defined

## Step 9: Create SKU Mapping DataFrame

In [ ]:
# Create SKU Hierarchy DataFrame
df_sku_mapping = pd.DataFrame([
    {'SKU': sku, **info} for sku, info in product_mapping.items()
])

print(f"✓ SKU Hierarchy mapping: {len(df_sku_mapping)} SKUs")
print("\nSample SKU Hierarchy:")
print(df_sku_mapping.head(10))

## Step 10: Write All Sheets to Excel Output

In [ ]:
print("Creating Excel output file...\n")

try:
    # Create Excel writer
    with pd.ExcelWriter(OUTPUT_FILE, engine='openpyxl') as writer:
        # Sheet 1: L2 Demand Detail
        df_l2_demand.to_excel(writer, sheet_name='L2_Demand_Detail', index=False)
        print("✓ Sheet 1: L2_Demand_Detail")
        
        # Sheet 2: L2 Demand Summary
        df_l2_summary.to_excel(writer, sheet_name='L2_Demand_Summary', index=False)
        print("✓ Sheet 2: L2_Demand_Summary")
        
        # Sheet 3: Product Family Summary
        df_family_summary.to_excel(writer, sheet_name='Product_Family_Summary', index=False)
        print("✓ Sheet 3: Product_Family_Summary")
        
        # Sheet 4: BOM Materials
        df_bom.to_excel(writer, sheet_name='BOM_Materials', index=False)
        print("✓ Sheet 4: BOM_Materials")
        
        # Sheet 5: BOM Summary
        df_bom_summary.to_excel(writer, sheet_name='BOM_Summary', index=False)
        print("✓ Sheet 5: BOM_Summary")
        
        # Sheet 6: SNP Planned Production
        df_snp_filtered.to_excel(writer, sheet_name='SNP_Planned_Production', index=False)
        print("✓ Sheet 6: SNP_Planned_Production")
        
        # Sheet 7: Procurement Analysis
        df_procurement.to_excel(writer, sheet_name='Procurement_Analysis', index=False)
        print("✓ Sheet 7: Procurement_Analysis")
        
        # Sheet 8: Product Coverage
        df_coverage.to_excel(writer, sheet_name='Product_Coverage', index=False)
        print("✓ Sheet 8: Product_Coverage")
        
        # Sheet 9: SKU Hierarchy
        df_sku_mapping.to_excel(writer, sheet_name='SKU_Hierarchy', index=False)
        print("✓ Sheet 9: SKU_Hierarchy")
    
    print(f"\n{'='*70}")
    print("✓ Excel file created successfully!")
    print(f"✓ Location: {OUTPUT_FILE}")
    print(f"{'='*70}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Step 11: Summary & Validation

In [ ]:
print("\n" + "="*70)
print("ANALYSIS COMPLETE - SUMMARY")
print("="*70 + "\n")

print("INPUT FILES:")
print(f"  1. Aug'25 L2 Demand Plan: {df_l2_demand['Market_SKU'].nunique()} unique SKUs")
print(f"  2. SNP Planned Production: {len(df_snp_filtered)} records")
print(f"  3. BOM Materials: {len(df_bom)} materials\n")

print("DATA EXTRACTED:")
print(f"  • L2 Demand Records: {len(df_l2_demand)}")
print(f"  • Unique Products: {df_l2_demand['Product_ID'].nunique()}")
print(f"  • Unique Families: {df_l2_demand['Product_Family'].nunique()}")
print(f"  • Date Range: {df_l2_demand['Month'].min()} to {df_l2_demand['Month'].max()}\n")

print("PRODUCT HIERARCHY:")
print(f"  • Total SKUs Mapped: {len(product_mapping)}")
print(f"  • Available Products: {len(PRODUCTS_AVAILABLE)}")
print(f"  • Unavailable Products: {len(PRODUCTS_NOT_AVAILABLE)}\n")

print("OUTPUT EXCEL SHEETS (9 total):")
print(f"  1. L2_Demand_Detail - {len(df_l2_demand)} rows")
print(f"  2. L2_Demand_Summary - {len(df_l2_summary)} rows")
print(f"  3. Product_Family_Summary - {len(df_family_summary)} rows")
print(f"  4. BOM_Materials - {len(df_bom)} rows")
print(f"  5. BOM_Summary - {len(df_bom_summary)} rows")
print(f"  6. SNP_Planned_Production - {len(df_snp_filtered)} rows")
print(f"  7. Procurement_Analysis - {len(df_procurement)} rows")
print(f"  8. Product_Coverage - {len(df_coverage)} rows")
print(f"  9. SKU_Hierarchy - {len(df_sku_mapping)} rows\n")

print("OUTPUT FILE:")
print(f"  📁 {OUTPUT_FILE}")
print(f"\n✓ Analysis complete and ready for download!")

## Step 12: Data Quality Checks

In [ ]:
print("\nDATA QUALITY CHECKS:\n")

# Check 1: Missing values
print("1. Missing Values in L2 Demand:")
missing_l2 = df_l2_demand.isnull().sum()
if missing_l2.sum() > 0:
    print(missing_l2[missing_l2 > 0])
else:
    print("   ✓ No missing values\n")

# Check 2: Data types
print("2. Data Types:")
print(f"   ✓ EA values: {df_l2_demand['EA'].dtype}")
print(f"   ✓ Batches values: {df_l2_demand['Batches'].dtype}\n")

# Check 3: Value ranges
print("3. Value Ranges:")
print(f"   • EA: min={df_l2_demand['EA'].min()}, max={df_l2_demand['EA'].max()}")
print(f"   • Batches: min={df_l2_demand['Batches'].min()}, max={df_l2_demand['Batches'].max()}")
print(f"   • Batch Size: min={df_l2_demand['Batch_Size'].min()}, max={df_l2_demand['Batch_Size'].max()}\n")

# Check 4: Product coverage
print("4. Product Coverage:")
for product in PRODUCTS_AVAILABLE:
    count = len(df_l2_demand[df_l2_demand['Product_ID'] == product])
    status = "✓" if count > 0 else "⚠"
    print(f"   {status} Product {product}: {count} records")

print("\n✓ Data quality checks complete!")